<a href="https://colab.research.google.com/github/kwb425/class-2025-spring/blob/main/class-2025-spring_0606-0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### MYSQL install on os

In [ ]:
# Install MySQL Server
!apt-get update
!apt-get install mysql-server

### MYSQL run on os

In [ ]:
# Start the MySQL service
!service mysql start
# Log in to MySQL as root
!mysql -u root

```mysql
DROP USER IF EXISTS 'test'@'localhost';
CREATE USER 'test'@'localhost' IDENTIFIED WITH 'mysql_native_password' BY '1234';
GRANT ALL PRIVILEGES ON *.* TO 'test'@'localhost' WITH GRANT OPTION;
FLUSH PRIVILEGES;
```

```mysql
DROP DATABASE IF EXISTS TestDataset;
CREATE DATABASE TestDataset;
CREATE TABLE TestDataset.TestTable (
    ID INT AUTO_INCREMENT PRIMARY KEY,
    Name VARCHAR(255),
	Email VARCHAR(255)
);
INSERT INTO TestDataset.TestTable (Name, Email) VALUES
('John', 'John@gmail.com'),
('Alice', 'Alice@gmail.com'),
('Bob', 'Bob@gmail.com');
exit;
```

### LLM loading

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth
    !pip install datasets huggingface_hub


In [ ]:
from unsloth import FastLanguageModel
from huggingface_hub import login

# login(token="key")
CUSTOM_MODEL, CUSTOM_TOKENIZER = FastLanguageModel.from_pretrained(
    model_name = "userID/my_lora_model",
    max_seq_length = 2048,
    load_in_4bit = True,
)


### MYSQL python api (basic ops)

In [ ]:
!pip install gradio mysql-connector-python

In [ ]:
import mysql.connector
from mysql.connector import Error

# Connection
def create_connection():
    try:
        connection = mysql.connector.connect(
            host="localhost",       # Replace with your MySQL host
            user="test",            # Replace with your MySQL username
            password="1234",        # Replace with your MySQL password
            database="TestDataset"  # Replace with your database name
        )
        return connection
    except Error as e:
        print("Error:", e)
        return None

# CRUD
def create_user(name, email):
    conn = create_connection()
    cursor = conn.cursor()
    query = "INSERT INTO TestTable (name, email) VALUES (%s, %s)"
    cursor.execute(query, (name, email))
    conn.commit()
    cursor.close()
    conn.close()
    return f"User {name} created!"

def read_users():
    conn = create_connection()
    cursor = conn.cursor()
    query = "SELECT * FROM TestTable"
    cursor.execute(query)
    rows = cursor.fetchall()
    cursor.close()
    conn.close()
    return rows

def update_user(user_id, name, email):
    conn = create_connection()
    cursor = conn.cursor()
    query = "UPDATE TestTable SET name = %s, email = %s WHERE id = %s"
    cursor.execute(query, (name, email, user_id))
    conn.commit()
    cursor.close()
    conn.close()
    return f"User {user_id} updated!"

def delete_user(user_id):
    conn = create_connection()
    cursor = conn.cursor()
    query = "DELETE FROM TestTable WHERE id = %s"
    cursor.execute(query, (user_id,))
    conn.commit()
    cursor.close()
    conn.close()
    return f"User {user_id} deleted!"


### MYSQL python api (login) & GRADIO api (chat)

In [ ]:
from transformers import TextStreamer
import io
import sys

def login_user(name, email):
    conn = create_connection()
    cursor = conn.cursor()
    query = "SELECT * FROM TestTable WHERE name = %s AND email = %s"
    cursor.execute(query, (name, email))
    user = cursor.fetchone()
    cursor.close()
    conn.close()
    if user:
        return (f"User {name} logged in!", True)
    else:
        return ("Login failed: user not found.", False)

def chat_with_llm(curr_message, history):
    text = CUSTOM_TOKENIZER.apply_chat_template(
        [{"role" : "user", "content" : curr_message}],
        tokenize = False,
        add_generation_prompt = True, # Must add for generation
        enable_thinking = False, # Disable thinking
    )

    # Append the user message to history
    history.append((curr_message, ""))

    # Capture the output from the TextStreamer
    original_stdout = sys.stdout
    sys.stdout = captured_output = io.StringIO()

    # Create a TextStreamer
    streamer = TextStreamer(CUSTOM_TOKENIZER, skip_prompt = True)

    # Generate with streaming
    _ = CUSTOM_MODEL.generate(
        **CUSTOM_TOKENIZER(text, return_tensors = "pt").to("cuda"),
        max_new_tokens = 256,
        temperature = 0.7, top_p = 0.8, top_k = 20,
        streamer = streamer,
    )

    # Process the captured output and yield chunks
    response = ""
    for chunk in captured_output.getvalue().split(): # You might need to adjust how you split the output into chunks
        response += chunk + " "
        history[-1] = (curr_message, response.strip())
        yield "", history

    # # Restore stdout
    # sys.stdout = original_stdout


def update_ui_after_login(message, success):
    return message, gr.Tabs(selected=5 if success else 4), gr.update(visible=success)


### Gradio final wrapping

In [ ]:
import gradio as gr

with gr.Blocks() as demo:
    logged_in = gr.State(False)

    with gr.Tabs() as main_tabs:
        with gr.Tab("Create", id=0):
            name = gr.Textbox(label="Name")
            email = gr.Textbox(label="Email")
            create_btn = gr.Button("Create")
            create_output = gr.Textbox()
            create_btn.click(fn=create_user, inputs=[name, email], outputs=create_output)

        with gr.Tab("Read", id=1):
            read_btn = gr.Button("Read All Users")
            read_output = gr.Dataframe()
            read_btn.click(fn=read_users, inputs=[], outputs=read_output)

        with gr.Tab("Update", id=2):
            uid = gr.Number(label="User ID")
            uname = gr.Textbox(label="New Name")
            uemail = gr.Textbox(label="New Email")
            update_btn = gr.Button("Update")
            update_output = gr.Textbox()
            update_btn.click(fn=update_user, inputs=[uid, uname, uemail], outputs=update_output)

        with gr.Tab("Delete", id=3):
            did = gr.Number(label="User ID")
            delete_btn = gr.Button("Delete")
            delete_output = gr.Textbox()
            delete_btn.click(fn=delete_user, inputs=[did], outputs=delete_output)

        with gr.Tab("Login", id=4):
            login_name = gr.Textbox(label="Name")
            login_email = gr.Textbox(label="Email")
            login_btn = gr.Button("Login")
            login_output = gr.Textbox()

        with gr.Tab("Chat", id=5, visible=False) as chat_tab:
            chatbot = gr.Chatbot()
            msg = gr.Textbox()
            clear = gr.Button("Clear")
            msg.submit(chat_with_llm, [msg, chatbot], [msg, chatbot])
            clear.click(lambda: None, None, chatbot, queue=False)

    login_btn.click(
        fn=login_user,
        inputs=[login_name, login_email],
        outputs=[login_output, logged_in]
    ).then(
        fn=update_ui_after_login,
        inputs=[login_output, logged_in],
        outputs=[login_output, main_tabs, chat_tab]
    )



demo.launch(debug=True)
